# Decision Tree

## Introduction

Tree-based methods partition the feature space into a set of rectangles, and then fit a simple model (like a constant) in
each one. They are conceptually simple yet powerful.

## Regression Tree (CART)

To simplify matters, we will restrict our attention to recursive binary partitions. We first split the space into two regions,
and model the response by the mean of Y in each region. We choose the variable and split-point to achieve the best fit, then
one or both of these split regions are split into two more regions, and this process is continued until some stopping rules are met.

A key advantage of the recursive binary tree is its interpretability. The feature space partition is fully described by a single tree

### Goal

Assume we have observed N sample pairs $\{x_i, y_i\}$, where $x_i = [x_{i1}, ..., x_{id}]^T$ has d dimensions. Our goal is to
create an algorithm that automatically decide on the splitting variables and splitting points.

## Solution

Suppose we end up having M regions (leaves) $R_1, R_2,..., R_M$ and we model the prediction in each region as a constant $c_m$

$\hat f (x_i) = \sum_{m=1}^{M} c_m I(x_i \in R_m)$

This equation sum over all region and only the region that contains $x_i$ will output $c_m$

If we have loss function $L = \sum_{i=1}^{N} (y_i - \hat f (x_i))^2 = \sum_{i=1}^{N} (y_i - \sum_{m=1}^{M} c_m I(x_i \in R_m))^2$

In order to minimize this loss function with respect to $c_j$, we take the derivative of L w.r.t $c_j$

$\implies \frac{\partial L}{\partial c_j} = 2\sum_{i=1}^{N} (y_i - \sum_{m=1}^{M} c_m I(x_i \in R_m)) (I(x_i \in R_j)) = 0$

$\implies \underbrace{\sum_{i=1}^{N} y_i I(x_i \in R_j)}_{\text{sum of all } y_i \text{ in } R_j} = \sum_{i=1}^{N}\sum_{m=1}^{M} c_m I(x_i \in R_m) I(x_i \in R_j)$

$\implies \underbrace{\sum_{i=1}^{N} y_i I(x_i \in R_j)}_{\text{sum of all } y_i \text{ in } R_j} = \sum_{i=1}^{N}\sum_{m=1}^{M} c_m I(x_i \in R_m) I(x_i \in R_j)$

If we look at right hand side equation, $I(x_i \in R_j)$ can determine the value of $\sum_{m=1}^{M} c_m I(x_i \in R_m)$
$\implies \sum_{i=1}^{N}\sum_{m=1}^{M} c_m I(x_i \in R_m) I(x_i \in R_j) = \sum_{i=1}^{N} c_j I(x_i \in R_j) I(x_i \in R_j) =
\sum_{i=1}^{N} c_j I(x_i \in R_j)$ Since:

  \begin{equation}
    \sum_{m=1}^{M} c_m I(x_i \in R_m) =
    \begin{cases}
      c_j I(x_i \in R_j), &  I(x_i \in R_j) = 1\\
      0, & I(x_i \in R_j) = 0
    \end{cases}
  \end{equation}

$\implies \hat c_j = \frac{\overbrace{\sum_{i=1}^{N} y_i I(x_i \in R_j)}^{\text{sum of all } y_i \text{ in } R_j}}
{\underbrace{\sum_{i=1}^{N} c_j I(x_i \in R_j)}_{\text{all the samples in } R_j}}$ = the mean of $y \in R_j$ = $\bar R_j$

However, finding the best binary partition in terms of minimize sum of squares is generally infeasible, hence, we proceed with a greedy algorithm

Starting with all data, consider a splitting variable j and splitting point s. Then:

$R_1 (j, s) = {x_i; x_{ij} \leq s}$, $R_2 (j, s) = {x_i; x_{ij} > s}$

Then we seek the splitting variable j and splitting point s that solve:

$\text{min}_{j, s} [\text{min}_{c_1} \sum_{x_i \in R_1 (j, s)} (y_i - \hat f (x_i))^2 + \text{min}_{c_2} \sum_{x_i \in R_2 (j, s} (y_i - \hat f (x_i))^2 ]$

For any j, s pair, the inner minimization is solved by

$\hat c_1 = \bar R_1$, $\hat c_2 = \bar R_2$ as showed above.

Each j, s pair can be done very quickly and hence by scanning all the possible j, s pair is feasible. Then this process is repeated until
stopping criteria met.

In [166]:
import numpy as np
import pandas as pd
from collections import Counter

class DecisionTreeNode:

    def __init__(self, split_point=None, split_feature=None, gini_index=None,
                 num_instance=None, left_child=None, right_child=None, prediction=None):

        self.split_point = split_point
        self.split_feature = split_feature
        self.gini_index = gini_index
        self.num_instance = num_instance
        self.left_child = left_child
        self.right_child = right_child
        self.prediction = prediction


class DecisionTreeClassifier:

    def __init__(self, max_depth=10, criterion='gini_index', min_sample_leave=1):

        self.max_depth = max_depth
        self.criterion = criterion
        self.min_sample_splits = min_sample_leave
        self._tree = None
        self._num_features = None
        self.feature_names = None
        self._num_samples = None
        self._classes = None

    def fit(self, x_train, y_train):

        self._num_samples, self._num_features = x_train.shape
        self._classes = np.unique(y_train)

        if isinstance(x_train, pd.DataFrame):
            self.feature_names = x_train.columns
            x_train = x_train.values

        else:
            self.feature_names = range(self._num_features)

        self._tree = self._grow_tree(np.column_stack([x_train, y_train]))

        return self

    def _grow_tree(self, train, curr_depth=0, stop=False):

        if stop:

            return None

        else:

            impurity_dict = {}

            for j in range(self._num_features):
                curr_feature_col = train[:, j]

                for s in np.unique(curr_feature_col):
                    impurity = self._cal_impurity(train, j, s)

                    if impurity in impurity_dict.keys():
                        impurity_dict[impurity].append((j, s))

                    else:
                        impurity_dict[impurity] = [(j, s)]

            min_impurity = min(impurity_dict.keys())
            j_hat, s_hat = impurity_dict[min_impurity][0]
            prediction = Counter(train[:, -1]).most_common()[0][0]
            R_l = train[train[:, j_hat] < s_hat]
            R_r = train[train[:, j_hat] >= s_hat]
            sample_split = [len(R_l), len(R_r)]

            if_stop = self._check_stopping_criterion(curr_depth, sample_split)

            print(curr_depth, min_impurity, sample_split, (j_hat, s_hat))

            return DecisionTreeNode(split_point=s_hat,
                                    split_feature=j_hat,
                                    prediction=prediction,
                                    gini_index=min_impurity,
                                    num_instance=len(train),
                                    left_child=self._grow_tree(train=R_l,
                                                               curr_depth=curr_depth+1,
                                                               stop=if_stop),

                                    right_child=self._grow_tree(train=R_r,
                                                                curr_depth=curr_depth+1,
                                                                stop=if_stop))

    def _check_stopping_criterion(self, curr_depth, sample_split):

        if curr_depth > self.max_depth:

            return True

        if any([sample_split[0] < self.min_sample_splits, sample_split[1] < self.min_sample_splits]):

            return True

        return False

    def _cal_impurity(self, train, j, s):

        R_l = train[train[:, j] < s]
        R_r = train[train[:, j] >= s]
        R_l_y = R_l[:, -1]
        R_r_y = R_r[:, -1]
        N_l = len(R_l) + 1e-10
        N_r = len(R_r) + 1e-10


        if self.criterion == 'gini_index':

            gini_l = 0
            gini_r = 0

            for k in self._classes:
                p_l = len(R_l_y[R_l_y == k]) / N_l
                p_r = len(R_r_y[R_r_y == k]) / N_r
                gini_l += p_l * (1 - p_l)
                gini_r += p_r * (1 - p_r)

            return gini_l * N_l + gini_r * N_r

    def predict(self, x_test):

        output = []

        for i in x_test:
            output.append(self._traverse_tree(i))

        return output


    def _traverse_tree(self, x, tree=None):

        if not tree:
            tree = self._tree

        if not tree.left_child and not tree.left_child:
            return tree.prediction

        else:

            if x[tree.split_feature] < tree.split_point:
                return self._traverse_tree(x, tree.left_child)

            else:
                return self._traverse_tree(x, tree.right_child)

In [167]:
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris


In [168]:
x = load_iris()['data']
y = load_iris()['target']

In [169]:
dt = DecisionTreeClassifier()

In [170]:
dt.fit(x, y)

0 50.00000000015 [50, 100] (2, 3.0)
1 1.0000333894311098e-10 [0, 50] (0, 4.3)
1 11.030595813383455 [54, 46] (3, 1.8)
2 4.625000000151477 [48, 6] (2, 5.0)
3 2.000039023286604e-10 [47, 1] (3, 1.7)
4 1.0000389405462329e-10 [0, 47] (0, 4.9)
4 1.000000082740371e-10 [0, 1] (0, 4.9)
3 1.333333333488889 [3, 3] (3, 1.6)
4 1.000000082740371e-10 [0, 3] (0, 6.0)
4 2.000000165480742e-10 [2, 1] (0, 7.2)
5 1.000000082740371e-10 [0, 2] (0, 6.0)
5 1.000000082740371e-10 [0, 1] (0, 7.2)
2 1.3333333334888893 [3, 43] (2, 4.9)
3 2.000000165480742e-10 [1, 2] (0, 6.0)
4 1.000000082740371e-10 [0, 1] (0, 5.9)
4 1.000000082740371e-10 [0, 2] (0, 6.0)
3 1.0000011929633958e-10 [0, 43] (0, 5.6)


In [171]:
dt.predict(x)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0]

In [64]:
x[x[:, 0] > 4.3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.2],
       [5. , 3

In [154]:
from sklearn.tree import DecisionTreeClassifier as dtc

In [155]:
sklearn_dt = dtc(criterion='gini')

In [151]:
a = sklearn_dt.fit(x, y)

In [152]:
x = np.random.randn(150, 20)

In [153]:
dt.fit(x, y)

0 95.50827423179226 [141, 9] (2, 1.5124405135712635)
1 89.95726495734502 [24, 117] (2, -0.9091603853372097)
2 8.800000000141335 [15, 9] (12, 0.24230862804764847)
3 6.200000000138 [10, 5] (1, 0.7974179820174572)
4 2.857142857302041 [3, 7] (7, -0.9508707242673596)
5 1.000000082740371e-10 [0, 3] (0, -1.2723087876537915)
5 2.000000165480742e-10 [2, 5] (15, -0.5560257755504455)
6 1.000000082740371e-10 [0, 2] (0, -0.4056771007718066)
6 1.000000082740371e-10 [0, 5] (0, -0.5602849539531976)
4 1.000000082740371e-10 [0, 5] (0, -1.6302814629478992)
3 1.000000082740371e-10 [0, 9] (0, -1.1840128923662574)
2 72.90178571437612 [96, 21] (9, 1.1895231209830655)
3 60.047619047715415 [84, 12] (17, 0.97065890806597)
4 50.65789473697546 [76, 8] (14, 1.4806081327956586)
5 47.454545454624025 [54, 22] (15, 0.8265259881411527)
6 31.836734694012577 [49, 5] (4, 1.6549885831156617)
7 27.969696969784422 [22, 27] (12, 0.04786008006349329)
8 7.107142857263967 [14, 8] (6, 0.7523571341720127)
9 1.9999979450346927e-10 

In [157]:
a.predict()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [172]:
accuracy_score(dt.predict(x), y)

1.0